In [2]:
!pip install Catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.1 MB/s 


In [3]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
import xgboost
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [4]:
df = pd.read_csv('justice.csv')

In [5]:
df.head()

,Unnamed: 0,ID,name,href,docket,term,first_party,second_party,facts,facts_len,majority_vote,minority_vote,first_party_winner,decision_type,disposition,issue_area
0,0,50606,Roe v. Wade,https://api.oyez.org/cases/1971/70-18,70-18,1971,Jane Roe,Henry Wade,"<p>In 1970, Jane Roe (a fictional name used in...",501,7,2,True,majority opinion,reversed,NaN
1,1,50613,Stanley v. Illinois,https://api.oyez.org/cases/1971/70-5014,70-5014,1971,"Peter Stanley, Sr.",Illinois,<p>Joan Stanley had three children with Peter ...,757,5,2,True,majority opinion,reversed/remanded,Civil Rights
2,2,50623,Giglio v. United States,https://api.oyez.org/cases/1971/70-29,70-29,1971,John Giglio,United States,<p>John Giglio was convicted of passing forged...,495,7,0,True,majority opinion,reversed/remanded,Due Process
3,3,50632,Reed v. Reed,https://api.oyez.org/cases/1971/70-4,70-4,1971,Sally Reed,Cecil Reed,"<p>The Idaho Probate Code specified that ""male...",378,7,0,True,majority opinion,reversed/remanded,Civil Rights
4,4,50643,Miller v. California,https://api.oyez.org/cases/1971/70-73,70-73,1971,Marvin Miller,California,"<p>Miller, after conducting a mass mailing cam...",305,5,4,True,majority opinion,vacated/remanded,First Amendment


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3303 entries, 0 to 3302
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          3303 non-null   int64 
 1   ID                  3303 non-null   int64 
 2   name                3303 non-null   object
 3   href                3303 non-null   object
 4   docket              3303 non-null   object
 5   term                3303 non-null   object
 6   first_party         3302 non-null   object
 7   second_party        3302 non-null   object
 8   facts               3303 non-null   object
 9   facts_len           3303 non-null   int64 
 10  majority_vote       3303 non-null   int64 
 11  minority_vote       3303 non-null   int64 
 12  first_party_winner  3288 non-null   object
 13  decision_type       3296 non-null   object
 14  disposition         3231 non-null   object
 15  issue_area          3161 non-null   object
dtypes: int64(5), object(11)


In [7]:
data = df[['facts', 'disposition', 'decision_type', 'issue_area', 'first_party_winner']]

In [8]:
data.head()

,facts,disposition,decision_type,issue_area,first_party_winner
0,"<p>In 1970, Jane Roe (a fictional name used in...",reversed,majority opinion,NaN,True
1,<p>Joan Stanley had three children with Peter ...,reversed/remanded,majority opinion,Civil Rights,True
2,<p>John Giglio was convicted of passing forged...,reversed/remanded,majority opinion,Due Process,True
3,"<p>The Idaho Probate Code specified that ""male...",reversed/remanded,majority opinion,Civil Rights,True
4,"<p>Miller, after conducting a mass mailing cam...",vacated/remanded,majority opinion,First Amendment,True


In [9]:
data = data.dropna()

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3099 entries, 1 to 3302
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   facts               3099 non-null   object
 1   disposition         3099 non-null   object
 2   decision_type       3099 non-null   object
 3   issue_area          3099 non-null   object
 4   first_party_winner  3099 non-null   object
dtypes: object(5)
memory usage: 145.3+ KB


In [12]:
#nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
def remove_stopwords(row):
    sentence = row['facts']
    cleaned = []
    sentence = re.sub(r'<[a-z]+>', '', sentence)
    sentence = re.sub(r'\n', '', sentence)
    for punct in string.punctuation:
        sentence = sentence.replace(punct, '')
    for word in sentence.split():
        if word not in stop_words:
            cleaned.append(word)
    return ' '.join(cleaned).lower()

In [14]:
data['cleaned_facts'] = data.apply(remove_stopwords, axis = 1)

In [15]:
data[['facts', 'cleaned_facts']]

,facts,cleaned_facts
1,<p>Joan Stanley had three children with Peter ...,joan stanley three children peter stanley the ...
2,<p>John Giglio was convicted of passing forged...,john giglio convicted passing forged money ord...
3,"<p>The Idaho Probate Code specified that ""male...",the idaho probate code specified males must pr...
4,"<p>Miller, after conducting a mass mailing cam...",miller conducting mass mailing campaign advert...
5,<p>Ernest E. Mandel was a Belgian professional...,ernest e mandel belgian professional journalis...
...,...,...
3297,<p>For over a century after the Alaska Purchas...,for century alaska purchase 1867 federal gover...
3298,"<p>Refugio Palomar-Santiago, a Mexican nationa...",refugio palomarsantiago mexican national grant...
3299,<p>Tarahrick Terry pleaded guilty to one count...,tarahrick terry pleaded guilty one count posse...
3300,<p>Joshua James Cooley was parked in his picku...,joshua james cooley parked pickup truck side r...


In [16]:
print(data['facts'].iloc[0])
print('----')
print(data['cleaned_facts'].iloc[0])

<p>Joan Stanley had three children with Peter Stanley.  The Stanleys never married, but lived together off and on for 18 years.  When Joan died, the State of Illinois took the children.  Under Illinois law, unwed fathers were presumed unfit parents regardless of their actual fitness and their children became wards of the state.  Peter appealed the decision, arguing that the Illinois law violated the Equal Protection Clause of the Fourteenth Amendment because unwed mothers were not deprived of their children without a showing that they were actually unfit parents.  The Illinois Supreme Court rejected Stanley’s Equal Protection claim, holding that his actual fitness as a parent was irrelevant because he and the children’s mother were unmarried.</p>

----
joan stanley three children peter stanley the stanleys never married lived together 18 years when joan died state illinois took children under illinois law unwed fathers presumed unfit parents regardless actual fitness children became wa

In [17]:
data

,facts,disposition,decision_type,issue_area,first_party_winner,cleaned_facts
1,<p>Joan Stanley had three children with Peter ...,reversed/remanded,majority opinion,Civil Rights,True,joan stanley three children peter stanley the ...
2,<p>John Giglio was convicted of passing forged...,reversed/remanded,majority opinion,Due Process,True,john giglio convicted passing forged money ord...
3,"<p>The Idaho Probate Code specified that ""male...",reversed/remanded,majority opinion,Civil Rights,True,the idaho probate code specified males must pr...
4,"<p>Miller, after conducting a mass mailing cam...",vacated/remanded,majority opinion,First Amendment,True,miller conducting mass mailing campaign advert...
5,<p>Ernest E. Mandel was a Belgian professional...,reversed,majority opinion,First Amendment,True,ernest e mandel belgian professional journalis...
...,...,...,...,...,...,...
3297,<p>For over a century after the Alaska Purchas...,reversed/remanded,majority opinion,Civil Rights,True,for century alaska purchase 1867 federal gover...
3298,"<p>Refugio Palomar-Santiago, a Mexican nationa...",reversed/remanded,majority opinion,Criminal Procedure,True,refugio palomarsantiago mexican national grant...
3299,<p>Tarahrick Terry pleaded guilty to one count...,affirmed,majority opinion,Criminal Procedure,False,tarahrick terry pleaded guilty one count posse...
3300,<p>Joshua James Cooley was parked in his picku...,vacated/remanded,majority opinion,Civil Rights,True,joshua james cooley parked pickup truck side r...


In [18]:
final_df = data.drop(['facts'], axis = 1)

In [19]:
lr = LogisticRegression()
dt = DecisionTreeClassifier()
gb = GradientBoostingClassifier()
rf = RandomForestClassifier()
xgb = xgboost.XGBClassifier()
cbt = CatBoostClassifier()
svc = SVC()
models = [lr, dt, gb, rf, xgb, cbt, svc]
names = ['Logistic Regression','DecisionTree','GradientBoosting','RandomForest','XGB','CATBOOST', 'support vector machine']

def evaluate_models(xTrain, yTrain, xVal, yVal, feature = ""):
    xs = [xTrain, yTrain, xVal, yVal]
    random_seed= 0
    scores = []
    for i in range(len(models) - 1):
        model = models[i +1] 
        model.fit(xTrain, yTrain)
        y_predicted = model.predict(xVal)
        accuracy = accuracy_score(yVal, y_predicted)
        f1score = f1_score(yVal, y_predicted)
        precision = precision_score(yVal, y_predicted)
        recall = recall_score(yVal, y_predicted)
        scores.append({
            'model':names[i] + ' with ' + feature ,
            'model_score':model.score(xTrain, yTrain),
            'accuracy' : accuracy,
            'f1_score' : f1score,
            'precision': precision,
            'recall' : recall
        })
    return pd.DataFrame(scores, columns=['model','model_score','accuracy','f1_score', 'precision', 'recall'])

In [20]:
def get_cv(X, y):
    scores = []
    for i in range(len(models)):
        model = models[i]
        cv = np.mean(np.absolute(cross_val_score(model, X, y, cv = 5)))
        scores.append({'model' : names[i],
                      'CV' : cv})
    return pd.DataFrame(scores, columns = ['model', 'CV'])

In [21]:
X = final_df.reset_index().drop(['index','first_party_winner'], axis = 1)
y = final_df.reset_index()['first_party_winner'].map({True:1, False:0})

In [22]:
for col in X.columns:
    if col != 'cleaned_facts':
        uniq_val = {v : i for i,v in enumerate(np.unique(X[col]))}
        print(uniq_val)
        X[col] = X[col].map(uniq_val)

{'affirmed': 0, 'none': 1, 'reversed': 2, 'reversed in-part': 3, 'reversed in-part/remanded': 4, 'reversed/remanded': 5, 'vacated': 6, 'vacated in-part/remanded': 7, 'vacated/remanded': 8}
{'dismissal - improvidently granted': 0, 'dismissal - moot': 1, 'dismissal - other': 2, 'equally divided': 3, 'majority opinion': 4, 'opinion of the court': 5, 'per curiam': 6, 'plurality opinion': 7}
{'Attorneys': 0, 'Civil Rights': 1, 'Criminal Procedure': 2, 'Due Process': 3, 'Economic Activity': 4, 'Federal Taxation': 5, 'Federalism': 6, 'First Amendment': 7, 'Interstate Relations': 8, 'Judicial Power': 9, 'Miscellaneous': 10, 'Privacy': 11, 'Private Action': 12, 'Unions': 13}


In [23]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.3)

In [24]:
X

,disposition,decision_type,issue_area,cleaned_facts
0,5,4,1,joan stanley three children peter stanley the ...
1,5,4,3,john giglio convicted passing forged money ord...
2,5,4,1,the idaho probate code specified males must pr...
3,8,4,7,miller conducting mass mailing campaign advert...
4,2,4,7,ernest e mandel belgian professional journalis...
...,...,...,...,...
3094,5,4,1,for century alaska purchase 1867 federal gover...
3095,5,4,2,refugio palomarsantiago mexican national grant...
3096,0,4,2,tarahrick terry pleaded guilty one count posse...
3097,8,4,1,joshua james cooley parked pickup truck side r...


# Using Bag of words approach

In [25]:
cv = CountVectorizer()

In [26]:
bowX = cv.fit_transform(X['cleaned_facts'])

In [27]:
final_df.shape

(3099, 5)

In [28]:
bowX.toarray().shape

(3099, 25115)

In [29]:
bowX = pd.DataFrame(bowX.toarray())

In [30]:
bowX_f = pd.concat([bowX, X.drop(['cleaned_facts'], axis = 1)], axis = 1)

In [31]:
bowX_f

,0,1,2,3,4,5,6,7,8,9,...,25108,25109,25110,25111,25112,25113,25114,disposition,decision_type,issue_area
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,4,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,4,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,4,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,4,7
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,4,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3094,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,4,1
3095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,4,2
3096,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,2
3097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,4,1


In [ ]:
get_cv(bowX_f, y)

C:\Users\RosaY\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\RosaY\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

[21:56:47] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\RosaY\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:07:03] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\RosaY\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

465:	learn: 0.0954618	total: 4m 27s	remaining: 5m 6s
466:	learn: 0.0953884	total: 4m 28s	remaining: 5m 6s
467:	learn: 0.0953152	total: 4m 28s	remaining: 5m 5s
468:	learn: 0.0952542	total: 4m 29s	remaining: 5m 5s
469:	learn: 0.0951853	total: 4m 30s	remaining: 5m 5s
470:	learn: 0.0951134	total: 4m 31s	remaining: 5m 4s
471:	learn: 0.0950516	total: 4m 31s	remaining: 5m 4s
472:	learn: 0.0949853	total: 4m 32s	remaining: 5m 3s
473:	learn: 0.0949200	total: 4m 33s	remaining: 5m 3s
474:	learn: 0.0948264	total: 4m 33s	remaining: 5m 2s
475:	learn: 0.0947573	total: 4m 34s	remaining: 5m 2s
476:	learn: 0.0946972	total: 4m 35s	remaining: 5m 1s
477:	learn: 0.0946292	total: 4m 35s	remaining: 5m 1s
478:	learn: 0.0945672	total: 4m 36s	remaining: 5m
479:	learn: 0.0945031	total: 4m 37s	remaining: 5m
480:	learn: 0.0944251	total: 4m 37s	remaining: 4m 59s
481:	learn: 0.0943436	total: 4m 38s	remaining: 4m 59s
482:	learn: 0.0942717	total: 4m 38s	remaining: 4m 58s
483:	learn: 0.0942027	total: 4m 39s	remaining: 4m

618:	learn: 0.0828706	total: 5m 54s	remaining: 3m 38s
619:	learn: 0.0828093	total: 5m 54s	remaining: 3m 37s
620:	learn: 0.0827466	total: 5m 55s	remaining: 3m 36s
621:	learn: 0.0826891	total: 5m 55s	remaining: 3m 36s
622:	learn: 0.0826285	total: 5m 56s	remaining: 3m 35s
623:	learn: 0.0825658	total: 5m 56s	remaining: 3m 35s
624:	learn: 0.0824430	total: 5m 57s	remaining: 3m 34s
625:	learn: 0.0823534	total: 5m 57s	remaining: 3m 33s
626:	learn: 0.0822917	total: 5m 58s	remaining: 3m 33s
627:	learn: 0.0822295	total: 5m 58s	remaining: 3m 32s
628:	learn: 0.0821692	total: 5m 59s	remaining: 3m 31s
629:	learn: 0.0821098	total: 5m 59s	remaining: 3m 31s
630:	learn: 0.0820516	total: 6m	remaining: 3m 30s
631:	learn: 0.0819912	total: 6m	remaining: 3m 30s
632:	learn: 0.0819214	total: 6m 1s	remaining: 3m 29s
633:	learn: 0.0818638	total: 6m 1s	remaining: 3m 28s
634:	learn: 0.0818014	total: 6m 2s	remaining: 3m 28s
635:	learn: 0.0817416	total: 6m 2s	remaining: 3m 27s
636:	learn: 0.0813414	total: 6m 3s	remai

771:	learn: 0.0692015	total: 7m 24s	remaining: 2m 11s
772:	learn: 0.0691535	total: 7m 25s	remaining: 2m 10s
773:	learn: 0.0691061	total: 7m 26s	remaining: 2m 10s
774:	learn: 0.0690346	total: 7m 26s	remaining: 2m 9s
775:	learn: 0.0689867	total: 7m 27s	remaining: 2m 9s
776:	learn: 0.0689390	total: 7m 28s	remaining: 2m 8s
777:	learn: 0.0688373	total: 7m 28s	remaining: 2m 7s
778:	learn: 0.0687894	total: 7m 29s	remaining: 2m 7s
779:	learn: 0.0687423	total: 7m 29s	remaining: 2m 6s
780:	learn: 0.0686950	total: 7m 30s	remaining: 2m 6s
781:	learn: 0.0686478	total: 7m 30s	remaining: 2m 5s
782:	learn: 0.0686031	total: 7m 31s	remaining: 2m 5s
783:	learn: 0.0685575	total: 7m 32s	remaining: 2m 4s
784:	learn: 0.0685104	total: 7m 33s	remaining: 2m 4s
785:	learn: 0.0684651	total: 7m 33s	remaining: 2m 3s
786:	learn: 0.0684193	total: 7m 34s	remaining: 2m 2s
787:	learn: 0.0683729	total: 7m 34s	remaining: 2m 2s
788:	learn: 0.0683262	total: 7m 35s	remaining: 2m 1s
789:	learn: 0.0682294	total: 7m 36s	remaini

925:	learn: 0.0596266	total: 8m 56s	remaining: 42.9s
926:	learn: 0.0595893	total: 8m 57s	remaining: 42.3s
927:	learn: 0.0595528	total: 8m 57s	remaining: 41.7s
928:	learn: 0.0594721	total: 8m 58s	remaining: 41.2s
929:	learn: 0.0594351	total: 8m 59s	remaining: 40.6s
930:	learn: 0.0593735	total: 8m 59s	remaining: 40s
931:	learn: 0.0593356	total: 9m	remaining: 39.4s
932:	learn: 0.0592995	total: 9m 1s	remaining: 38.9s
933:	learn: 0.0592620	total: 9m 1s	remaining: 38.3s
934:	learn: 0.0592246	total: 9m 2s	remaining: 37.7s
935:	learn: 0.0591886	total: 9m 3s	remaining: 37.1s
936:	learn: 0.0591514	total: 9m 3s	remaining: 36.6s
937:	learn: 0.0591142	total: 9m 4s	remaining: 36s
938:	learn: 0.0590777	total: 9m 5s	remaining: 35.4s
939:	learn: 0.0589926	total: 9m 5s	remaining: 34.8s
940:	learn: 0.0589554	total: 9m 6s	remaining: 34.3s
941:	learn: 0.0589184	total: 9m 7s	remaining: 33.7s
942:	learn: 0.0588820	total: 9m 8s	remaining: 33.1s
943:	learn: 0.0588448	total: 9m 9s	remaining: 32.6s
944:	learn: 0

,model,CV
0,Logistic Regression,0.899646
1,DecisionTree,0.953853
2,GradientBoosting,0.968375
3,RandomForest,0.953208
4,XGB,0.969666
5,CATBOOST,0.969990
6,support vector machine,0.964500


In [32]:
bow_train = pd.DataFrame(cv.fit_transform(Xtrain['cleaned_facts']).toarray())
bow_train = pd.concat([bow_train, Xtrain.reset_index().drop(['index','cleaned_facts'], axis = 1)], axis = 1)

bow_test = pd.DataFrame(cv.transform(Xtest['cleaned_facts']).toarray())
bow_test = pd.concat([bow_test, Xtest.reset_index().drop(['index','cleaned_facts'], axis = 1)], axis = 1)

In [33]:
bow_train

,0,1,2,3,4,5,6,7,8,9,...,20490,20491,20492,20493,20494,20495,20496,disposition,decision_type,issue_area
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,4,9
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,6,7
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,4,4
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,4,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2164,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,4,9
2165,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,4,9
2166,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,4,10
2167,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,4


In [34]:
bow_test

,0,1,2,3,4,5,6,7,8,9,...,20490,20491,20492,20493,20494,20495,20496,disposition,decision_type,issue_area
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,4,6
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,4
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,4,4
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,4,1
926,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,4,2
927,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,4,4
928,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,4


In [35]:
evaluate_models(bow_train, ytrain, bow_test, ytest, feature = 'Bag of words')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklea

Learning rate set to 0.014339
0:	learn: 0.6699493	total: 154ms	remaining: 2m 33s
1:	learn: 0.6477973	total: 237ms	remaining: 1m 58s
2:	learn: 0.6265022	total: 318ms	remaining: 1m 45s
3:	learn: 0.6060615	total: 405ms	remaining: 1m 40s
4:	learn: 0.5864819	total: 484ms	remaining: 1m 36s
5:	learn: 0.5677491	total: 570ms	remaining: 1m 34s
6:	learn: 0.5498200	total: 653ms	remaining: 1m 32s
7:	learn: 0.5326333	total: 728ms	remaining: 1m 30s
8:	learn: 0.5160007	total: 802ms	remaining: 1m 28s
9:	learn: 0.5002992	total: 887ms	remaining: 1m 27s
10:	learn: 0.4852354	total: 965ms	remaining: 1m 26s
11:	learn: 0.4708812	total: 1.04s	remaining: 1m 25s
12:	learn: 0.4571395	total: 1.13s	remaining: 1m 25s
13:	learn: 0.4439331	total: 1.21s	remaining: 1m 25s
14:	learn: 0.4312662	total: 1.28s	remaining: 1m 24s
15:	learn: 0.4191485	total: 1.36s	remaining: 1m 23s
16:	learn: 0.4074964	total: 1.44s	remaining: 1m 23s
17:	learn: 0.3963366	total: 1.52s	remaining: 1m 22s
18:	learn: 0.3857137	total: 1.61s	remaining:

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


,model,model_score,accuracy,f1_score,precision,recall
0,Logistic Regression with Bag of words,1.000000,0.965591,0.975309,0.966361,0.984424
1,DecisionTree with Bag of words,0.973260,0.974194,0.981453,0.973926,0.989097
2,GradientBoosting with Bag of words,1.000000,0.966667,0.976172,0.963581,0.989097
3,RandomForest with Bag of words,0.975565,0.975269,0.982239,0.973966,0.990654
4,XGB with Bag of words,0.973721,0.975269,0.982239,0.973966,0.990654
5,CATBOOST with Bag of words,0.974182,0.969892,0.978295,0.973765,0.982866


In [39]:
accuracy_score(ytest, models[4].predict(bow_test))

0.975268817204301

In [36]:
import pickle as pk

In [41]:
with open('bow_cv.pk', 'wb') as f:
  pk.dump(cv, f)

with open('model.pk', 'wb') as f:
  pk.dump(models[4], f)